Notebook to compute the eigenmode decomposition of 1D diffusion utilizing 
random walk methodology. Used to validate modal markov formulation for calcium
diffusion. Simple 1d diffusion with no reactions.

11 nodes.

author: Margot Wagner
date created: 6/16/22

### setup

In [1]:
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt
import math
from typing import Union, Tuple

In [2]:
# constants
# create mesh
n_spatial_locs = 11  # define number of grid points along 1D line
dt = 1  # time step (usec)
n_time_pts = 1000  # number of time points

particle_start_loc = 5  # starting position for molecules
min_loc = 0  # minimum position
max_loc = n_spatial_locs - 1  # maximum position
last_elem_i = -1  # index for the last element in a list

line_length = 4  # total length of diffusion line (um)
n_particles = 1  # number of molecules
diffusion_constant_D = 2.20e-4  # Calcium diffusion coeff (um^2/usec)

### 1D Random Walk Diffusion Simulation

(implementation similar to GeeksforGeeks' ["Random Walk (Implementation in Python)"](https://www.geeksforgeeks.org/random-walk-implementation-python/))

In [3]:
def get_jump_probability(
    line_length: Union[int,float], 
    n_spatial_locs: int, 
    diffusion_constant_D: float, 
    dt: Union[int,float]
) -> Tuple[float, float]:
    """Find the probability of moving one spot to the left or right based on 
    finite-difference approximations

    params:
        line_length:
            length of line on which molecule is diffusing
        n_spatial_locs:
            number of locations molecule can diffuse to
        diffusion_constant_D:
            calcium diffusion constant
        dt:
            time step

    return:
        - probability of diffusing one spot to the left or right (k*dt)
        - k: diffusion rate constant

    """
    dx = line_length / n_spatial_locs  # distance of one "hop"
    diffusion_rate_constant_k = diffusion_constant_D / dx**2  # rate constant

    return diffusion_rate_constant_k * dt, diffusion_rate_constant_k

In [4]:
def random_walk_simulation(n_particles, n_time_pts, jump_probability):
    """1-D random walk for n_particles from a range of 
    positions = [0, (n_spatial_locs - 1)]

    Args:
      n_particles:
        total number of molecules
      n_time_pts:
        number of time points
      jump_probability:
        probability of particle diffusing to neighboring grid point

    Returns:
      positions of all particles over time - matrix shaped 
        (n_particles, n_time_pts)
    """

    # probabilities depending on if particle is in middle or at edge
    jump_probability_middle = [
        jump_probability,
        1 - 2 * jump_probability,
        jump_probability,
    ]
    jump_probability_edge = [jump_probability, 1 - jump_probability]

    # initialize array for all particle positions
    particle_locs = np.empty((n_particles, n_time_pts), dtype="int64")

    for n in range(n_particles):

        # Initialize starting position (0 to (n_spatial_locs - 1) range)
        positions = [particle_start_loc]

        # sampling probability all at once (1000 timepoints)
        rand = np.random.random(n_time_pts - 1)

        # movement decision conditions
        move_l_cond = rand < jump_probability
        move_r_cond = rand > (1 - jump_probability)
        # stay condition is between the two

        # run simulation for particle n
        # check probability rolls
        for move_left, move_right in zip(move_l_cond, move_r_cond):

            # move left if move_left=True and last position != minimum 
            # position 
            left = move_left and positions[last_elem_i] > min_loc

            # move right if move_right=True and last position != maximum 
            # position 
            right = move_right and positions[last_elem_i] < max_loc

            # stay condition is implied

            # adjust position accordingly
            positions.append(positions[last_elem_i] - left + right)

        # add results to cumulative array
        particle_locs[n] = positions

    return particle_locs

In [5]:
def random_walk_postprocess(particle_locs, plot=False):
    """Post-process 1D random-walk diffusion 
    (counts, normalized counts, means)

    Args:
      particle_locs:
        positions of all particles over time - matrix shaped 
        (n_particles, n_time_pts)

    Returns:
      unnorm_n_per_loc:
        number of particles in each position over time (unnormalized)
      n_per_loc:
        number of particles in each position over time (normalized)
      mean_n_per_loc:
        mean value of particle number at each location over the whole 
        simulation
    """

    # distribution of particles across positions over time
    unnorm_n_per_loc = np.zeros(
        (n_spatial_locs, n_time_pts), dtype="int64"
    )  # number of particles over time
    n_per_loc = np.zeros(
        (n_spatial_locs, n_time_pts)
    )  # normalized count over time

    for i in range(n_time_pts):
        # count number of particles in each position
        counts = np.bincount(particle_locs[:, i])

        # resize to include all positions if it doesn't already
        counts.resize(n_spatial_locs)

        # assign number of particles
        unnorm_n_per_loc[:, i] = counts

        # normalize counts and assign
        counts = counts / particle_locs.shape[0]
        n_per_loc[:, i] = counts

    mean_n_per_loc = np.mean(unnorm_n_per_loc, axis=1)

    if plot:
        # plot particle counts for each position
        plt.figure(figsize=(14, 10))

        for i in range(n_spatial_locs):
            plt.plot(list(range(n_time_pts)), n_per_loc[i, :])

        plt.title(
            "Normalized number of particles in each position over time",
            fontsize=20,
        )
        plt.xlabel("timepoint", fontsize=14)
        plt.ylabel("normalized count", fontsize=14)
        plt.legend(list(range(n_spatial_locs)))
        plt.show()

    return unnorm_n_per_loc, n_per_loc, mean_n_per_loc

### Eigenmode Markov Model

In [6]:
def transition_matrix_maker(diffusion_rate_constant_k, n_spatial_locs):
    """Builds and returns the transition matrix for the 1D random walk case 
    as given.

    params:
        diffusion_rate_constant_k:
            jump rate constant
        n_spatial_locs:
            number of grid points along line

    returns:
        transition matrix
    """

    # Define A (transition) matrix
    A = np.zeros(
        (n_spatial_locs, n_spatial_locs)
    )  # transition probability between grid points

    # Transition matrix is given by the ODE dynamics equation (using k-values)
    vec_diag = np.full(n_spatial_locs, (-2 * diffusion_rate_constant_k))
    vec_off_diag = np.full(
        (n_spatial_locs - 1), diffusion_rate_constant_k
    )  # off-diagonal values

    # create transition matrix
    A = (
        np.diag(vec_diag, k=0)
        + np.diag(vec_off_diag, k=1)
        + np.diag(vec_off_diag, k=-1)
    )
    A[0, 0] = -diffusion_rate_constant_k
    A[n_spatial_locs - 1, n_spatial_locs - 1] = -diffusion_rate_constant_k

    return A

When the particle is placed at the node x, initial conditions are given by:
\begin{equation} 
    m_{\pm k} = \frac{1}{2} \pm \frac{n_{x, t0} \text{v}_{x k}}{2\sqrt{n_{x, t0} (\text{v}_{x k})^2}} 
\end{equation}

For number of particles, $n_{x, t0}$ = 1:
\begin{equation} 
    m_{\pm k} = \frac{1}{2} \pm \frac{1}{2} 
\end{equation}

In [7]:
def get_eigenvalues_and_vectors(A, print_output=True, plot_eigenmodes=False):
    """Returns the sorted eigenvalues and eigenvectors of matrix A

    Args:
        A: transition matrix

    Returns:
        eigenvalues: 1d matrix of size n_spatial_locs
        eigenvectors: 2d matrix of eigenvectors where columns
                correspond to eigenvalues (ie evec[:,k] <-> eval[k])
        eval_sort_index: argsort index array used to sort eigenvalues.
                Can be used to sort via matrix[eval_sort_index]

    """

    # get eigenvalues/eigenvectors
    # eigenmode[k] is composed of eigenvector[:, k] and eigenvalue[k]
    e_val_unsorted, e_vec_unsorted = eig(A)
    np.set_printoptions(suppress=True)  # gets rid of scientific notation

    if print_output:
        print("EIGENVALUES")
        print(" ", end="")

        # sort values
        eigenvalues = np.sort(e_val_unsorted)
        eval_sort_index = np.argsort(e_val_unsorted)
        eigenvalues[0] = round(eigenvalues[0])
        [print(i, end=" " * 5) for i in range(n_spatial_locs)]
        print()
        print(eigenvalues.round(decimals=3))
        print()
        print("EIGENVECTORS")
        # eigenvector columns correspond to eigenvalues 
        # (ie evec[:,k] <-> eval[k])
        print("   ", end="")
        [print(i, end=" " * 4) for i in range(n_spatial_locs)]
        print()
        eigenvectors = e_vec_unsorted[:, eval_sort_index]
        # normalize eigenvector values
        eigenvectors = eigenvectors / eigenvectors[0, 0]
        print(eigenvectors.round(decimals=1))
        print()

    if plot_eigenmodes:
        print("EIGENMODES (e^(-eigenvalue * t))")
        ## TODO: ADD LEGEND
        plot_eigenmodes(eigenvalues)

    return eigenvalues, eigenvectors, eval_sort_index

In [8]:
def get_eigenmode(eigenvalues, t):
    return np.exp(-eigenvalues * t)


def plot_eigenmodes(eigenvalues):
    time = np.array(range(n_time_pts))

    for i in range(len(eigenvalues)):
        plt.plot(time, get_eigenmode(eigenvalues[i], time))

    plt.show()

In [9]:
def eigenmode_init_conditions(
    eigenvectors, eval_sort_index, print_output=True
):
    """Find the initial normalized number of particles in the positive 
    and negative state of eigenmode k

    params:
        eigenvectors:
            eigenvector columns correspond to eigenvalues 
            (ie evec[:,k] <-> eval[k])
            eigenvector[e, eigenmode (k)]

    return:
        normalized number of particles in each eigenmode at time = 0
    """

    # starting loc given by particle_start_loc

    # new index of starting node location in sorted eigenvalue/vector arrays
    start_loc_eigenvalue_i = np.where(eval_sort_index == particle_start_loc)

    # get eigenvector for starting location, all eigenmodes
    start_loc_eigenvector = eigenvectors[start_loc_eigenvalue_i, :]

    # variable fraction component of initial conditions equation
    fraction_value = (n_particles * start_loc_eigenvector) / (
        2 * np.sqrt(n_particles * (start_loc_eigenvector**2))
    )

    n_per_positive_mode = 0.5 + fraction_value
    n_per_negative_mode = 0.5 - fraction_value

    if print_output:
        print("EIGENMODE INITIAL CONDITIONS")
        print("POSITIVE")
        print(n_per_positive_mode)
        print("NEGATIVE")
        print(n_per_negative_mode)
        print()

    return n_per_positive_mode, n_per_negative_mode

In [10]:
def get_eigenmode_transition_probability(eigenvalues, print_output=True):

    transition_probability = (eigenvalues / 2) * dt

    if print_output:
        print("EIGENMODE TRANSITION PROBABILITIES")
        [print(i, end="\t") for i in range(n_spatial_locs)]
        print()
        [
            print("{:0.1e}".format(transition_probability[i]), end=" ")
            for i in range(n_spatial_locs)
        ]
        print()

    return transition_probability

In [11]:
def eigenmode_markov_simulation(
    n_eigenmodes,
    init_cond,
    transition_probability,
    n_particles,
    dt,
    n_time_pts,
    plot=False,
):
    """Markov simulation for eigenmode analysis to capture calcium diffusion 
    over time
    
    params:
        n_eigenmodes:
            number of modes (k not +/- k); equal to number of locations 
            (n_spatial_loc)
        init_cond:
            initial distribution of particles between positive and negative 
            eigenmode states; [positive vector, negative vector]
        transition_probability:
            probability of transitioning between + and - eigenmode states
        n_particles:
            number of particles
        dt:
            timestep
        n_time_pts:
            number of timepoints

    return:
        n_per_eigenmode_state: normalized number of particles in each 
        eigenmode (+/-) at each timepoint
        np aray shape (n_modes x n_time x n_eigenmode_states)
    """
    # positive and negative states
    n_eigenmode_states = 2

    # initialize number of particles
    # n_modes x n_time x n_eigenmode_states (for +/-, this is 2)
    n_per_eigenmode_state = np.zeros(
        (n_spatial_locs, n_time_pts, n_eigenmode_states)
    ).astype("int")

    # assign initial conditions scaling for number of molecules
    for j in range(n_eigenmode_states):
        n_per_eigenmode_state[:, 0, j] = init_cond[j] * n_particles

    # for each time point
    for i in range(n_time_pts - 1):

        # for each eigenmode
        for k in range(n_spatial_locs):

            # initialize the number of particles that transition
            # [from + -> -, from - -> +]
            n_change = [0, 0]

            # find number of transitions positive/negative eigenmode state;
            for j in range(n_eigenmode_states):

                # sample random numbers equal to number of particles in 
                # current state
                r = np.random.random(n_per_eigenmode_state[k, i, j])

                # sum number of particles that transitioned
                n_change[j] = sum(r < transition_probability[k])

            # update next time point
            for j in range(n_eigenmode_states):
                n_per_eigenmode_state[k, i + 1, j] = (
                    n_per_eigenmode_state[k, i, j]
                    - n_change[j]
                    + n_change[1 - j]
                )

    if plot:
        n_plot_columns = 2
        n_plot_rows = math.ceil(n_spatial_locs / n_plot_columns)
        fig, ax = plt.subplots(n_plot_columns, n_plot_rows, figsize=(14, 10))

        m_count = 0
        for i in range(n_plot_columns):
            for j in range(n_plot_rows):
                for k in range(n_eigenmode_states):
                    if m_count < n_spatial_locs:
                        ax[i, j].plot(
                            list(range(n_time_pts)),
                            n_per_eigenmode_state[m_count, :, k],
                        )
                        ax[i, j].set_title("Eigenmode {}".format(m_count))

                m_count += 1

        # fig.suptitle
        fig.tight_layout()
        plt.show()

    return n_per_eigenmode_state / n_particles

To transform the number of particles in each eigenmode to the number at each spatial location, we use the following equation:

(from my notes)

\begin{equation} 
    \eta_{kt} (m_{+kt} - m_{-kt}) = m_{kt} = \sum_i^{N_{particles}} n_{it} \text{v}_{ik}
\end{equation}

\begin{equation} 
    \sqrt{\sum_i^{N_{particles}} n_{it} \text{v}_{i k}^2} (m_{+kt} - m_{-kt}) =  \sum_i^{N_{particles}} n_{it} \text{v}_{ik}
\end{equation}

\begin{equation} 
    m_k = \sum_i n_i \text{v}_{i k}
\end{equation}

(from Gert's notes)


\begin{equation}
\begin{split}
    \sqrt{\sum_i^{N_{particles}} n_{it} \text{v}_{i k}^2} (m_{+kt} - m_{-kt}) = \sum_i^{N_{particles}} m_i \text{v}_{i k}^2 \\
    \sum_i^{N_{particles}} n_{it} \text{v}_{i k}^2 = \left( \frac{\sum_i^{N_{particles}} m_i \text{v}_{i k}^2}{(m_{+kt} - m_{-kt})} \right)^2\\
    
\end{split}
\end{equation}

n: number of particles in each spatial node

m: number of particles in each eigenmode

v: eigenvectors

**We want to obtain n_{it}**



In [12]:
def eigenmodes_to_spatial_nodes(n_per_eigenmode_state, eigenvectors):
    """Calculate the number of particles at each node from the eigenmode 
    representation.

    Args:
        n_per_eigenmode_state: normalize the number of particles in each node;
        np aray shape (n_modes x n_time x n_eigenmode_states)
        eigenvectors: eigenvector of node i (vector); v[:,k] is the eigenvector
        corresponding to the eigenvalue w[k]


    Returns:
        np array containing normalized particle counts for each node 
        (n_nodes x n_time_pts)
    """

    # initialize node values (n_nodes x n_time_pts)
    node_vals_from_modes = np.zeros((n_spatial_locs, n_time_pts))
    
    # positive - negative
    (n_per_eigenmode_state[:,:,0] - n_per_eigenmode_state[:,:,1])
    
    print(n_per_eigenmode_state[:,:,0] * (eigenvectors**2))
    
    # for each time point, for a given eigenmode, sqrt of sum (across number of nodes) of the number particles at node i times the eigenvector
    
    

    #for i in range(n_spatial_locs):
    #    e_vec_i = eigenvectors[:, i].reshape((n_spatial_locs, 1))
#
    #    # equation from definition
    #    node_vals_from_modes[i, :] = np.sum(
    #        (e_vec_i * n_per_eigenmode_state), axis=0
    #    )

    return node_vals_from_modes

In [13]:
def graph_num_comparison(n1, n2, legend_vals=[], title=""):
    """Graph comparing normalized number of particles at each node 
    and in each mode.

    Args:
        n1: normalized number of particles in one condition 
        (particles per time)
        n2: normalized number of particles in one condition 
        (particles per time)
    """

    half_n_nodes = int(n1.shape[0] / 2)

    fig, axes = plt.subplots(half_n_nodes, 2, figsize=(14, 10))

    for i, ax in enumerate(axes.flat):
        ax.plot(list(range(n_time_pts)), n1[i, :])
        ax.plot(list(range(n_time_pts)), n2[i, :])
        ax.legend(legend_vals)
        ax.set_title("{} {}".format(title, i))

    # plt.legend(["nodes", "modes"])
    fig.tight_layout()
    plt.show()

### Main

In [14]:
def main():

    # find probability of moving one step
    jump_probability, jump_rate_constant_k = get_jump_probability(
        line_length, n_spatial_locs, diffusion_constant_D, dt
    )

    # run simulation
    particle_locs = random_walk_simulation(
        n_particles, n_time_pts, jump_probability
    )

    # plot output
    unnorm_n_per_loc, n_per_loc, mean_n_per_loc = random_walk_postprocess(
        particle_locs, plot=False
    )

    # build transition matrix
    A = transition_matrix_maker(-jump_rate_constant_k, n_spatial_locs)

    # get sorted eigenvalues and eigenvectors (normalized)
    eigenvalues, eigenvectors, eval_sort_index = get_eigenvalues_and_vectors(
        A, print_output=True, plot_eigenmodes=False
    )

    # Find initial values for the number of particles in each eigenmode
    # For n_particles = 1, it should just be 1 and 0 respectively
    init_n_positive_modes, init_n_negative_modes = eigenmode_init_conditions(
        eigenvectors, eval_sort_index
    )

    # initialize eigenmode Markov model transition probabilities
    eigenmode_transition_probability = get_eigenmode_transition_probability(
        eigenvalues, print_output=True
    )

    # run Markov model
    n_per_eigenmode_state = eigenmode_markov_simulation(
        n_spatial_locs,
        [init_n_positive_modes, init_n_negative_modes],
        eigenmode_transition_probability,
        n_particles,
        dt,
        n_time_pts,
        plot=False,
    )
    
    # TODO: to go from modes to nodes, an additional factor needs to be included
    #   this factor is in Gert's notes. I also took a screenshot and put it in
    #   my notes app. So yeah, do that. But be careful that you are using the 
    #   eigenvectors correctly given the transpose issue above
    # difference between + and - 
    
    # Knowns from simulation: q (+/-) k (aka m (+/-) k) [number of particles per eigenmode internal state]
    # Previous knowns: eigenvectors and eigenvalues
    
    # Compare to: p or n (nodes)  [number of particles per node]
    node_vals_from_modes = eigenmodes_to_spatial_nodes(n_per_eigenmode_state, eigenvectors)






"""
    q_per_mode = (q_per_mode_pn_split[:,:,0] - q_per_mode_pn_split[:,:,1])
    q_per_mode = np.sqrt(denom_sum)*(q_per_mode_pn_split[:,:,0] 
                        - q_per_mode_pn_split[:,:,1])


    # find node particle values from mode
    node_vals_from_modes = nodes_from_modes(q_per_mode, e_vec)

    # comparison of initial nodes and node from mode outputs
    #graph_num_comparison(p_per_loc, node_vals_from_modes, 
    # legend_vals=["rw nodes", "mode to node"], title="Node")
"""


if __name__ == "__main__":
    tables_created = main()

EIGENVALUES
 0     1     2     3     4     5     6     7     8     9     10     
[0.    0.    0.001 0.001 0.002 0.003 0.004 0.005 0.006 0.006 0.007]

EIGENVECTORS
   0    1    2    3    4    5    6    7    8    9    10    
[[ 1.  -1.4 -1.4 -1.3 -1.2 -1.1  0.9 -0.8  0.6  0.4  0.2]
 [ 1.  -1.3 -0.9 -0.4  0.2  0.8 -1.2  1.4 -1.4 -1.1 -0.6]
 [ 1.  -1.1 -0.2  0.8  1.4  1.3 -0.6 -0.4  1.2  1.4  0.9]
 [ 1.  -0.8  0.6  1.4  0.9 -0.4  1.4 -1.1 -0.2 -1.3 -1.2]
 [ 1.  -0.4  1.2  1.1 -0.6 -1.4  0.2  1.3 -0.9  0.8  1.4]
 [ 1.  -0.   1.4  0.  -1.4 -0.  -1.4  0.   1.4 -0.  -1.4]
 [ 1.   0.4  1.2 -1.1 -0.6  1.4  0.2 -1.3 -0.9 -0.8  1.4]
 [ 1.   0.8  0.6 -1.4  0.9  0.4  1.4  1.1 -0.2  1.3 -1.2]
 [ 1.   1.1 -0.2 -0.8  1.4 -1.3 -0.6  0.4  1.2 -1.4  0.9]
 [ 1.   1.3 -0.9  0.4  0.2 -0.8 -1.2 -1.4 -1.4  1.1 -0.6]
 [ 1.   1.4 -1.4  1.3 -1.2  1.1  0.9  0.8  0.6 -0.4  0.2]]

EIGENMODE INITIAL CONDITIONS
POSITIVE
[[[1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0.]]]
NEGATIVE
[[[0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.]]]

EIGENMODE T

ValueError: operands could not be broadcast together with shapes (11,1000) (11,11) 

For n_particles = 1
\begin{equation} 
    q_{\pm k} = \frac{1}{2} \pm \frac{ \text{v}_{k}}{2\sqrt{ (\text{v}_{ k})^2}} 
\end{equation}

\begin{equation} 
    u_ik = \sqrt{\sum_i p_i \text{v}_{ik}^2}
\end{equation}

For n_particles = 1
\begin{equation} 
    q_{\pm k} = \frac{1}{2} \pm \frac{\sum_i p_i \text{v}_{i k}}{2\sqrt{\sum_i p_i (\text{v}_{i k})^2}} 
\end{equation}